## Retrieving Relevant Context using Word Embeddings

### Work Breakdown

- [x] Find pre-trained embeddings in Bulgarian
- [x] Load wikipedia documents
- [x] Chunk documents into sentences
    - [ ] Maybe it would be better to start with passing full documents, in order to keep the context?
- [x] Load embedding vectors
    - [x] Build an index from word to vector for mapping words to vectors
    - [x] Build an index from vector to vector for nearest neighbour search
- [x] Map each sentence/passage to an embedding vector
    - [ ] Paremeterize the aggregation function - avg, max, etc.
- [x] Load questions and answer options from `bg_rc-v1.0.json`
- [x] For each question find the nearest contexts (passages, sentences)
- [x] For each question add to it the list of most relevant texts to the questions and answers under the "context" key
- [ ] Paremeterize each step and expose configuration for grid search

In [1]:
import json
import numpy as np
import pandas as pd
import ujson
import glob
import ngtpy
import codecs

# import elasticsearch
from elasticsearch import Elasticsearch
from polyglot.text import Text, Word

from tqdm.auto import tqdm

from IPython.utils import io

In [2]:
# custom modules
import importlib
import sys
sys.path.insert(0, '..')

import dataset
from dataset.bgquiz import BGQuiz
_ = importlib.reload(dataset.bgquiz)

In [3]:
tqdm.pandas()

In [4]:
es = Elasticsearch('localhost')

In [5]:

def query_es(es, query, index_name, num_hits=1, query_field='passage', highligh_size=400, num_highlights=3,
             explain=False):
    #     res = es.search(index=name, body={"query": {"match": { query_field: query }},
    #                                               'from': 0, 'size': num_hits})

    res = es.search(
        index=index_name,
        body={
            "explain": explain,
            "query": {
                "multi_match": {
                    "query": query,
                    "fields": query_field,
                }
            },
            "highlight": {
                "fragment_size": highligh_size,
                "type": "plain",
                "number_of_fragments": num_highlights,
                "fields": {
                    "passage": {}
                }
            },
            "from": 0,
            "size": num_hits
        })
    #     return res
    #     scores = [hit['_score'] for hit in res['hits']['hits']]

    return [{
        "score":
            x['_score'],
        "source":
            x['_source'],
        "highlight":
            x['highlight']['passage'] if hasattr(x, 'highlight') else ''
    } for x in res['hits']['hits']]


In [27]:
config = {
    'documents_glob': '../data/context/bgwiki.json/*/wiki_*',
    'embeddings_path': '../data/word2vec/model.txt',
    'embeddings_index_name': 'word2vec',
    'questions_path': '../data/bg_rc-v1.0.json',
    
    'es_index_name': 'bgwiki_test',
    'es_query_field': [
        'title.bulgarian^2',
        'passage.ngram',
        'passage',
        'passage.bulgarian^2'
    ],
    'es_result_count': 50,
    'total_passages_per_question': 5,
}

# numberbatch_config = {
#     'documents_glob': '../data/context/bgwiki.json/*/wiki_*',
#     'embeddings_path': '../data/conceptnet-numberbatch/bg.txt',
#     'embeddings_index_name': 'numberbatch',
#     'questions_path': '../data/bg_rc-v1.0.json'
# }

In [8]:
def flatten(l):
    return [item for sublist in l for item in sublist]

In [9]:
# returns a dict which maps word to embedding vector
def load_embeddings(embeddings_filename):
    embeddings_index = {}

#     with open(embeddings_filename) as f:
    with codecs.open(embeddings_filename, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs

    dimensions = len(coefs)
    print('Found {} word vectors of {} dimensions.'.format(len(embeddings_index), dimensions))
    return embeddings_index, dimensions

In [10]:
embeddings_index, embedding_dim = load_embeddings(config['embeddings_path'])

Found 628026 word vectors of 100 dimensions.


In [11]:
def create_vector_space_index(vectors, index_name):
    dimensions = len(vectors[0])
    print('Creating a vector space index of {} vectors of {} dimensions...'.format(len(vectors), dimensions))
    ngtpy.create(index_name, dimensions)
    index = ngtpy.Index(index_name)
    index.batch_insert(vectors)
    index.save()
    print('Vector space index saved as: {}'.format(index_name))
    return index

In [31]:
# vector_index = create_vector_space_index(
#     vectors=list(embeddings_index.values()),
#     index_name=config['embeddings_index_name']
# )

Creating a vector space index of 20870 vectors of 300 dimensions...
Vector space index saved as: numberbatch


In [12]:
def tokenize_to_words(text):
    return list(Text(text).words)

def average_embeddings(word_embeddings, word_embeddings_dim, words):
    embeddings = [
        word_embeddings[word]
        for word in words
        if word in word_embeddings
    ]

    if len(embeddings) > 0:
        return np.average(embeddings, axis=0)
    else:
        return np.zeros(word_embeddings_dim)

def map_text_to_vector(word_embeddings, word_embeddings_dim, text):
    return average_embeddings(word_embeddings, word_embeddings_dim, tokenize_to_words(text))

In [13]:
def find_k_nearest_neighbours(vector_index, query_vector, k=20):
    return vector_index.search(query_vector, k)

In [20]:
# vector_index = create_vector_space_index(
#     vectors=df_chunked['vector'].tolist(),
#     index_name=config['embeddings_index_name']
# )

Creating a vector space index of 2502559 vectors of 100 dimensions...
Vector space index saved as: word2vec


In [35]:
def to_utf8(text):
    return ''.join(x for x in text if x.isprintable())

def augment_dataset(path):
    counter = 0
    with open(path, 'r') as f:
        json_data = json.load(f)

        for (category, items) in json_data['data'].items():
            for questions in items:
                for q in questions['questions']:
                    query_texts = [q['question']] + [answer for answer in q['answers']]

                    passages = set()
                    for a in q['answers']:
                        passages.update([
                            to_utf8(x['source']['passage'])
                            for x in query_es(
                                es,
                                q['question'] + ' ' + a,
                                config['es_index_name'],
                                config['es_result_count'],
                                config['es_query_field']
                            )
                        ])
                    
                    query_vectors = [
                        map_text_to_vector(embeddings_index, embedding_dim, query_text)
                        for query_text in query_texts
                    ]
                    
                    passage_vectors = [
                        map_text_to_vector(embeddings_index, embedding_dim, passage)
                        for passage in passages
                    ]
                    
                    vector_index = create_vector_space_index(
                        vectors=passage_vectors,
                        index_name='tmp'
                    )
                    
                    nearest_vectors = sorted(flatten([
                        find_k_nearest_neighbours(vector_index, query_vector)
                        for query_vector in query_vectors
                    ]), key=lambda x: x[1])[:config['total_passages_per_question']]
                    
                    passages_list = list(passages)
                    
                    most_relevant_texts = [
                        passages_list[index]
                        for index, distance in nearest_vectors
                    ]
                    
                    q['context'] = most_relevant_texts
                    
                    counter = counter + 1
                    if counter % 100 == 0:
                        print('Processed {} questions.'.format(counter))
        return json_data

In [36]:
augmented_json = augment_dataset(config['questions_path'])
# augment_dataset(config['questions_path'])

Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.


Creating a vector space index of 138 vectors of 100 dimensions...
Vector space index saved as: tmp


Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.


Creating a vector space index of 77 vectors of 100 dimensions...
Vector space index saved as: tmp


Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.


...

Creating a vector space index of 117 vectors of 100 dimensions...
Vector space index saved as: tmp


Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.


Creating a vector space index of 68 vectors of 100 dimensions...
Vector space index saved as: tmp


Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.
Detector is not able to detect the language reliably.


Creating a vector space index of 76 vectors of 100 dimensions...
Vector space index saved as: tmp


In [37]:
with open('../data/bg_rc_reranked_passages-v1.0.json', 'w', encoding='utf-8') as outfile:
    json.dump(augmented_json, outfile, ensure_ascii=False, indent=2)